In [12]:
import numpy as np

In [13]:
# get n digits of lfsr sequence for initial state register
# with connection poly cd over a field mod z
def get_lfsr_seq(register, cd, z, n):
    register = list(register)
    lfsr_out = []
    
    for i in range(n):
        feedback = 0
        for r, c, in zip(register, cd):
            feedback += r * c
        feedback = (z - feedback) % z
        
        lfsr_out.append(register.pop(0))
        register.append(feedback)
    
    return lfsr_out

In [14]:
keystream = '0001100100010100011110101101111010110000110011011011110101111001010011010101010100001000010111011011101000010000100110100000110111110110001001010011001010001000011010110010100011100011010001000'

# convert to a numpy array
keystream = np.array([int(k) for k in keystream])
n = len(keystream)

In [15]:
# connection polynomials expressed as z2 coefficients in
# descending degree
connection_polynomials = [
    [1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1],
    [1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0],
    [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0]]

In [16]:
# convert an int x to a binary list of length l
def to_binary_array(x, l):
    b = bin(x)[2:]
    r = [int(i) for i in b]
    while(len(r) < l):
        r = [0] + r
    return r

In [17]:
# measure similarity between 2 numpy arrays
def correlation_value(a, b):
    return np.count_nonzero(a == b) / len(a)

In [18]:
def crack_lfsr(cd):
    correlation_values = []
    initial_states = []
    l = len(cd)
    
    for i in range(2**l):
        # convert int i to initial state of the register
        i = to_binary_array(i, l)
        
        # get the sequence for initial state i
        seq = get_lfsr_seq(i, cd, 2, n)
        seq = np.array(seq)
        
        # measure similarity between lfsr with input i and output keystream
        c = abs(.5 - correlation_value(seq, keystream))
        
        correlation_values.append(c)
        initial_states.append(i)
        
    return (initial_states, correlation_values)

In [26]:
%%time
key = []
for cd in connection_polynomials:
    init, corr = crack_lfsr(cd)
    # add initial state with max correlation value to key
    cv = max(corr)
    state = init[corr.index(cv)]
    print(cv, state)
    key.append(state)

0.2979274611398963 [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1]
0.2979274611398963 [1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0]
0.22020725388601037 [0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]
CPU times: user 59.8 s, sys: 52 ms, total: 59.8 s
Wall time: 59.8 s


In [23]:
key

[[0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1],
 [1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0],
 [0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0]]

In [21]:
result = np.zeros(n)
for k, cd in zip(key, connection_polynomials):
    # add up sequences from lfsr with its cd and corresponding cracked key
    result = np.add(result, np.array(get_lfsr_seq(k, cd, 2, n)))

# final keystream is where sum of individual lfsr's >= 2
result = result >= 2

In [24]:
# make sure it is the same as the given keystream
(result == keystream).all()

True